# Visual odometry with Kimera package

Experiments with Kimera-VIO project.

## Kimera-VIO installation

See

0. Kimera home - https://github.com/MIT-SPARK/Kimera

1. Kimera image preparation: https://github.com/MIT-SPARK/Kimera-VIO/blob/master/docs/kimera_vio_install.md

2. Udocker for running containers in Colab - https://github.com/indigo-dc/udocker/blob/master/docs/installation_manual.md and https://gist.github.com/mwufi/6718b30761cd109f9aff04c5144eb885

3. Test data origin - https://drive.google.com/drive/folders/1-B6V-4S_ZsCqjZY5muXNxV_dqbmdeAto

4. Downloading files from Google Drive - https://colab.research.google.com/notebooks/io.ipynb

5. Downloading files [from ydisk](https://ru.stackoverflow.com/questions/1088300/%D0%BA%D0%B0%D0%BA-%D1%81%D0%BA%D0%B0%D1%87%D0%B8%D0%B2%D0%B0%D1%82%D1%8C-%D1%84%D0%B0%D0%B9%D0%BB%D1%8B-%D1%81-%D1%8F%D0%BD%D0%B4%D0%B5%D0%BA%D1%81-%D0%B4%D0%B8%D1%81%D0%BA%D0%B0).

6. Kimera evaluation - https://github.com/MIT-SPARK/Kimera-VIO-Evaluation

7. X window Virtual Frame Buffer - https://packages.debian.org/bullseye/xvfb

8. Python wrapper for Xvfb - https://github.com/ponty/PyVirtualDisplay

9. Example of running something with display in colab - https://codeease.net/programming/python/how-to-run-tkinter-in-google-colab

In [ ]:
!apt-get install -y xvfb
!pip install pyvirtualdisplay
!pip install pillow

In [ ]:
import os
if not os.path.exists("/home/user"):
    !useradd -m user
!pip install udocker
!udocker --allow-root install

## Data downloading

In [ ]:
import requests
from urllib.parse import urlencode


def download(url: str, output: str):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"
    final_url = base_url + urlencode(dict(public_key=url))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open(output, "wb") as f:
        f.write(download_response.content)

In [ ]:
download(url="https://disk.yandex.ru/d/Rd5d7BtgIFE3OQ", output="mh01.zip.001")
download(url="https://disk.yandex.ru/d/J6ryB7H8v-u1Ig", output="mh01.zip.002")

In [ ]:
!cat mh01.zip.* > ./mh01.zip

In [ ]:
!unzip -o ./mh01.zip > /dev/null

In [ ]:
!su - user -c "udocker pull docker.io/zaba247/kimera-vio"

In [ ]:
!su - user -c "udocker create --force --name=kimera docker.io/zaba247/kimera-vio"

In [ ]:
!if [ ! -d ./scripts ]; then mkdir ./scripts/; fi

In [ ]:
%%writefile ./scripts/run-inside-x.py
from pyvirtualdisplay import Display
import subprocess

display = Display(visible=0, size=(800, 600))
display.start()

output = "/output"
out = subprocess.run(["/bin/bash",
                    "/root/Kimera-VIO/scripts/stereoVIOEuroc.bash",
                    "-p", "/", "-log",
                    ], text=True,
                    capture_output=True)
display.stop()
print("================= stdout ===================")
print(out.stdout)
with open(f"{output}/stdout", "w") as file:
    file.write(out.stdout)
print("================= stderr ===================")
print(out.stderr)
with open(f"{output}/stderr", "w") as file:
    file.write(out.stderr)

In [ ]:
%%writefile ./scripts/stereoVIOEuroc.flags
#!/bin/bash
--log_output=true
--backend_type=1
--regular_vio_backend_modality=0
--deterministic_random_number_generator=true

# 2D Visualization
--visualize_feature_predictions=false
--visualize_feature_tracks=true

# 3D Visualization
--visualize=true
--visualize_lmk_type=false
--visualize_mesh=true
--visualize_mesh_2d=false
--visualize_mesh_with_colored_polygon_clusters=false
--visualize_point_cloud=true
--visualize_convex_hull=false
--visualize_plane_constraints=false
--visualize_planes=false
--visualize_plane_label=false
--visualize_semantic_mesh=false
--visualize_mesh_in_frustum=false
#--visualize_load_mesh_filename=/home/tonirv/datasets/euroc/V1_01_easy/mav0/pointcloud0/data.ply
# Type of 3D Visualization:
# 0: mesh 3d
# 1: pointcloud
# 2: none
--viz_type=1
--min_num_obs_for_mesher_points=3
--extract_planes_from_the_scene=false

# For displaying/saving Frontend images:
--visualize_frontend_images=false
--save_frontend_images=false
--log_feature_tracks=false
--log_mono_tracking_images=false
--log_stereo_matching_images=false

# output video from window
--final_k=100
--record_video_for_viz_3d=true

--add_extra_lmks_from_stereo=true

In [ ]:
%%writefile ./scripts/run-inside-docker.sh
apt-get update
apt-get install -y python3 xvfb x11-xserver-utils zip
pip install pyvirtualdisplay
pip install pillow
export DISPLAY=1
export QT_X11_NO_MITSHM=1
# changes options
cp -v /scripts/stereoVIOEuroc.flags /root/Kimera-VIO/params/Euroc/flags/stereoVIOEuroc.flags
python3 /scripts/run-inside-x.py
zip -r /output/kimera.zip /root/Kimera-VIO/

In [ ]:
!chmod +x ./scripts/run.sh ./scripts/run-inside-docker.sh
!if [ -d ./output ]; then rm -R ./output; fi
!mkdir ./output
!chmod 777 ./output
!ls -l ./

In [ ]:
!su - user -c "udocker --allow-root run -v '"`pwd`"/output:/output' kimera ls -l /root/Kimera-VIO/"

In [ ]:
!su - user -c "udocker --allow-root run -v '"`pwd`"/output:/output' -v '"`pwd`"/scripts/:/scripts' -v '"`pwd`"/MH_01_easy/mav0:/mav0' kimera /scripts/run-inside-docker.sh"

In [ ]:
!su - user -c "udocker --allow-root run kimera /root/Kimera-VIO/build/stereoVIOEuroc --help"